# Data Cleaning

Having acquired the data, this notebook will clean and preprocess text and extract new metadata.

In [19]:
import pandas as pd
import numpy as np
import re
from functools import reduce

In [2]:
df = pd.read_pickle('../data/interim/aggregated.pkl')

In [15]:
def find_handles(text):
    r = re.compile('@[^\s]+')
    return re.findall(r, text)

def find_hashtags(text):
    r = re.compile('#[^\s]+')
    return re.findall(r, text)

In [11]:
df['Tweeted At'] = df['Contents'].map(find_handles)

In [17]:
df['Hashtags'] = df['Contents'].map(find_hashtags)

### Create Hashtags DataFrame

In [22]:
hashtags = list(set(reduce(lambda x, y: x + y, list(df['Hashtags']))))

KeyboardInterrupt: 

In [ ]:
hashtags[:20]